Programmieren 3 - Grafik und GPU-Programmierung, Deep Learning

Peter Rösch, Fakultät für Informatik

Hochschule Augsburg, 2023/2024

# Die Mandelbrotmenge

Die Mandelbrotmenge wird an vielen Stellen beschrieben, einen Überblick finden Sie z.B. bei [Wikipedia](http://de.wikipedia.org/wiki/Mandelbrot-Menge).

$$
z_{n+1} = z^2_n + c, \; z_0 = 0, \; n < n_{\rm max}, \qquad {\rm mit}\quad z, c \in \mathbb{C}
$$

Wir wollen ein 2D-Bild berechnen, dessen Pixel jeweils einen bestimmten Wert für $ c\,$ repräsentieren. Die Grauwerte entsprechen dem größten Wert $n$, für den an dieser Stelle gilt: $n < n_{\rm max}$ und $|z_n| < 10$.

Wir bilden zunächst den Bereich $c \in [-2.5 - 1. 5i : 1.1 + 1.5 i]$ der komplexen Ebene auf ein Bild der Größe $1024 \times 1024$ ab. 

**Fragen:** 

1. Warum ist die Berechnung dieses Bildes parallelisierbar?
1. Wie sieht eine sinnvolle Aufteilung in Teilaufgaben aus?



## Jupyter: Vorbereitungen

In [ ]:
# numpy (Arrays)
import numpy as np
# matplotlib (Ausgabe der Ergebnisse)
import matplotlib.pyplot as plt
%matplotlib ipympl

# Groesse des zu berechnenden Bildes
N = 1024
s = (N, N)
# Bereich (Realteil von -2.5 bis 1.1, Imaginaerteil -1.5 bis 1.5)
rng = (-2.5, 1.1, -1.5, 1.5)

## CPU: Implementierung in Python

In [ ]:
# Quelle: Der Code basiert auf ein Beispiel von C. Rossant [1, Kap. 5]


def mandelbrot_python(a, c_min=-2.5 - 1.5j, c_max=1.1 + 1.5j, n_max=100):
    rng = c_max - c_min
    scale_r = rng.real / a.shape[1]
    scale_i = rng.imag / a.shape[0]
    for y in range(a.shape[0]):
        for x in range(a.shape[1]):
            c = complex(c_min.real + x * scale_r, c_min.imag + y * scale_i)
            z = 0
            a[y, x] = n_max - 1
            for n in range(n_max):
                z = z * z + c
                if abs(z) >= 10.0:
                    a[y, x] = n
                    break
    return (c_min.real, c_max.real, c_min.imag, c_max.imag), a

In [ ]:
# Test und Zeitmessung
a_Python = np.zeros(shape=s, dtype=np.uint8)
%timeit mandelbrot_python(a_Python)

In [ ]:
# Darstellung der Ergebnisse
ax = plt.figure(figsize=(10, 10)).add_subplot(111)
ax.imshow(a_Python, extent=rng, origin="lower", cmap=plt.cm.jet)

## CPU: Beschleunigung mit *numba*

In [ ]:
# Quelle: Der Code basiert auf ein Beispiel von C. Rossant [1, Kap. 5]
from numba import njit, prange


@njit(parallel=True)
def mandelbrot_numba(a, c_min=-2.5 - 1.5j, c_max=1.1 + 1.5j, n_max=100):
    rng = c_max - c_min
    scale_r = rng.real / a.shape[1]
    scale_i = rng.imag / a.shape[0]
    for y in prange(a.shape[0]):
        for x in range(a.shape[1]):
            c = complex(c_min.real + x * scale_r, c_min.imag + y * scale_i)
            z = 0
            a[y, x] = n_max - 1
            for n in range(n_max):
                z = z * z + c
                if abs(z) >= 10.0:
                    a[y, x] = n
                    break
    return (c_min.real, c_max.real, c_min.imag, c_max.imag), a

In [ ]:
# Test und Zeitmessung
a_numba = np.zeros(shape=s, dtype=np.uint8)
%timeit mandelbrot_numba(a_numba)

In [ ]:
# Darstellung der Ergebnisse
ax = plt.figure(figsize=(10, 10)).add_subplot(111)
ax.imshow(a_numba, extent=rng, origin="lower", cmap=plt.cm.jet)

# GPU: Einführung

Aktuelle Grafikkarten bieten zumindest theoretisch eine höhere Rechenleistung als die CPU.

**Nvidia GeForce RTX 4090:** 24GB, 1008 GB/s, 16384 Unified Shaders, ca. **82580/82580/1290 GFLOPS (FP16/FP32/FP64 prec.)**, 450 W (09/2022), siehe [TechPowerUp](https://www.techpowerup.com/gpu-specs/geforce-rtx-4090.c3889).

**AMD Ryzen™ Threadripper™ 3990X:** 64 Kerne, 128 Threads,  2.9 GHz (Base) 4.3 GHz (max. Boost), **13200 GFLOPS (FP 32)**, 280 W (02/2020), siehe [TechPowerUp](https://www.techpowerup.com/cpu-specs/ryzen-threadripper-3990x.c2271).
        
**Cray 2:** ca. **2 GFLOPS**, 200 kW (Supercomputer, 1985), siehe [Wikipedia](https://en.m.wikipedia.org/wiki/Cray-2).

## Wie bringt man diese "PS" auf die Straße?

Die Architektur der GPU unterscheidet sich deutlich von der einer CPU, siehe z.B. [NVIDA Ampere](https://developer.nvidia.com/blog/nvidia-ampere-architecture-in-depth/), Details finden Sie im [Ampere Whitepaper, S. 12, Fig.3](https://www.nvidia.com/content/dam/en-zz/Solutions/geforce/ampere/pdf/NVIDIA-ampere-GA102-GPU-Architecture-Whitepaper-V1.pdf).

Frage: Welche Auswirkungen hat das auf die Software-Entwicklung für GPUs?

## Mögliche Vorgehensweise

 1. Erstellung / Suchen einer CPU-Referenz-Implementierung
 1. Den Algorithmus parallelisierbar (um)formulieren
 1. Theoretische Vorüberlegungen zur Sinnhaftigkeit einer GPU-Implementierung (Bedarf an Zugriffen auf den globalen Speicher, Verhältnis Berechnungen/Speicherzugriffe, Identifikation von Flaschenhälsen)
 1. Entwurf des Kernels, der Caching-Strategie, Überlegungen zur Block- und Grid-Größe, Entwicklung einer Test-Strategie
 1. Implementierung und Test

    [Details (NVIDIA)](http://docs.nvidia.com/cuda/cuda-c-programming-guide).

## Rechnen auf der GPU - Schritte

 1. Initialisierung des Systems (OpenCL / CUDA / OpenGL / Vulkan).
 1. Compilation des Rechen-Kernels.
 1. Strukturierung der *threads* und *blocks* (*work-items* und *work-groups*).
 1. Kopieren der Eingabe-Daten z.B. auf die Grafikkarte.
 1. Aufruf des Kernels.
 1. Kopieren der Ergebnisse in den Hauptspeicher oder direkte Visualisierung der Resultate.

## Der Kernel ...

 * muss herausfinden, für welche Eingabedaten er zuständig ist.
 * muss sicherstellen, dass er nicht über Array-Grenzen hinaus liest und schreibt.
 * ist oft auch für die Umsetzung der Caching-Strategie zuständig.
 * setzt Mechanismen wie z.B. *barriers* und *atomic add* zur Synchronisation ein.
 * wird millionenfach gestartet.

# Mandelbrot auf der GPU

## Verwendung von numba mit CUDA

Quellen: [NVIDIA](https://developer.nvidia.com/blog/numba-python-cuda-acceleration), [numba-Dokumentation](https://numba.readthedocs.io/en/stable/cuda/kernels.html)

**Wichtig:** Falls keine NVIDIA-GPU vorhanden ist, kann der Cuda-Simulator von numba verwendet werden. Dazu muss die Umgebungsvariable [NUMBA_ENABLE_CUDASIM](https://numba.readthedocs.io/en/stable/reference/envvars.html) gesetzt werden.

In [ ]:
from numba import cuda

In [ ]:
@cuda.jit
def mandel_gpu(min_x, max_x, min_y, max_y, image, max_iters):
    x_index, y_index = cuda.grid(2)

    height, width = image.shape

    if x_index < width and y_index < height:
        pixel_size_x = (max_x - min_x) / width
        pixel_size_y = (max_y - min_y) / height

        x = min_x + x_index * pixel_size_x
        y = min_y + y_index * pixel_size_y

        c = complex(x, y)
        z = 0.0j
        image[y_index, x_index] = max_iters - 1
        for i in range(max_iters):
            z = z * z + c
            if (z.real * z.real + z.imag * z.imag) >= 100:
                image[y_index, x_index] = i
                break

In [ ]:
a_cuda = np.zeros(shape=s, dtype=np.uint8)
threadsperblock = np.array([8, 8], dtype=np.uint32)
blockspergrid = (a_cuda.shape + (threadsperblock - 1)) // threadsperblock
mandel_gpu[tuple(blockspergrid), tuple(threadsperblock)](
    -2.5, 1.1, -1.5, 1.5, a_cuda, 100
)
ax = plt.figure(figsize=(10, 10)).add_subplot(111)
ax.imshow(a_cuda, extent=rng, origin="lower", cmap=plt.cm.jet)

In [ ]:
%%timeit
mandel_gpu[tuple(blockspergrid), tuple(threadsperblock)]\
          (-2.5, 1.1, -1.5, 1.5, a_cuda, 100) 

Ergebnis (NVIDIA RTX A4000):

1.89 ms ± 6.15 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

# Klassifikation von Bildern


## Klassisches Beispiel: MNIST

**Aufgabe:** Erkennung handgeschriebener Ziffern aus Bildern. 
* Eingabe: Grauwertbild, Größe 28 x 28 Pixel
* Ausgabe: Ziffer (0-9).

**Aufgabe verstehen:** [Typische Eingabebilder](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png).

## MNIST mit Deep Learning

**Idee:** Merkmale (Features) und Regeln werde nicht explizit definiert und implementiert,
sondern über die Parameter eines Netzwerks gelernt. Einige Wissenschaftler warnen
vor der künstlichen Intelligenz, siehe z.B. [Will Knight in MIT technology review](https://www.technologyreview.com/s/604087/the-dark-secret-at-the-heart-of-ai).

Vorgehensweise: Maschinelles Lernen mittels neuronaler Netzwerke:
1. Definition des Netzwerks (Schichten, Verbindungen).
1. Erstellen großer annotierter Datensätze für Training und Evaluation.
1. Training mit dem Ziel, einen Parametersatz zu finden, der die Anzahl der
korrekten Klassifikationen maximiert.
1. Verwendung des Models und der durch Training gefundenen Parameter für die
Klassifikation unbekannter Datensätze.

**Links:**

Vortrag von Martin Görner: [online](https://cloud.google.com/blog/products/gcp/learn-tensorflow-and-deep-learning-without-a-phd)
    
Wikipedia: https://en.wikipedia.org/wiki/Deep_learning

Für die Bildverarbeitung (Wikipdia): [Convolutional Neural Networks](https://en.wikipedia.org/wiki/Convolutional_neural_network).    

## Werkzeuge (Auswahl)

**Tensorflow:** Von google entwickelte Open-source software-Bibliothek, die Python unterstützt und ab Version 2.0 auch [keras](https://keras.io) enthält. Die Installation mit
anaconda ist einfach: conda install tensorflow oder conda install
tensorflow-gpu, https://www.tensorflow.org

**PyTorch:** Von Facebook entwickelte Bibliothek für maschinelles Lernen, siehe [Web-Seite](https://pytorch.org).

**Spezialiserte Hardware:** [Google TPU](https://cloud.google.com/tpu), [Movedius Neural Compute Stick](https://newsroom.intel.com/news/intel-democratizes-deep-learning-application-development-launch-movidius-neural-compute-stick).

Vergleich verschiedener Werkzeuge: [Wikipedia](https://en.wikipedia.org/wiki/comparison_of_deep_learning_software).

## Keras

**Basis:** Tensorflow CNTK, Theano. In Tensorflow ab 2.x als tensorflow.keras enthalten.

**Dokumentation:** [online](https://keras.io)

**Relevanz:** Derzeit bei den [Kaggle Competitions](https://www.kaggle.com/competitions) erfolgreiches Framework.

**Idee:** Vereinfache Schnittstelle zur Erstellung der für Tensorflow charakteristischen
Datenflussgraphen.

**GPU:** unterstützung über Tensorflow, cuda und [cudnn](https://developer.nvidia.com/cudnn).

In [ ]:
"""
    module to load mnist data
    Author: F. Chollet
"""

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


def get_mnist_data():
    """
    Load and preprocess mnist data.

    Returns:
        (train_images, train_labels, test_images, test_labels): tuple
            normalised mnist data
    """
    (train_images, train_labels), (
        test_images,
        test_labels,
    ) = mnist.load_data()

    # reshape and normalise
    train_images = train_images.astype("float32") / 255
    train_labels = to_categorical(train_labels)

    test_images = test_images.astype("float32") / 255
    test_labels = to_categorical(test_labels)

    return (train_images, train_labels, test_images, test_labels)

Die Ausführung der nächsten Zelle sollte auf Rechnern ohne GPU übersprungen werden ...

In [ ]:
# Author: F. Chollet
# Die Ausführung auf der CPU dauert ...
if True:
    from tensorflow.keras import models
    from tensorflow.keras.models import save_model
    from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
    from tensorflow.keras.layers import Conv2D, MaxPooling2D

    from tensorflow.compat.v1 import ConfigProto
    from tensorflow.compat.v1 import InteractiveSession

    config = ConfigProto()
    config.gpu_options.allow_growth = True
    session = InteractiveSession(config=config)

    out_file_name = "deep.hdf5"
    batch_size = 512
    epochs = 10

    kernel_size = (3, 3)
    pool_size = (2, 2)
    nb_filters = 32
    input_shape = (28, 28, 1)

    network = models.Sequential()
    network.add(
        Conv2D(
            nb_filters,
            kernel_size,
            padding="valid",
            input_shape=input_shape,
            activation="relu",
        )
    )
    network.add(Conv2D(nb_filters, kernel_size, activation="relu"))
    network.add(MaxPooling2D(pool_size=pool_size))
    network.add(Dropout(0.25))
    network.add(Flatten())
    network.add(Dense(128, activation="relu"))
    network.add(Dropout(0.5))
    network.add(Dense(10, activation="softmax"))

    network.summary()

    network.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )

    train_images, train_labels, test_images, test_labels = get_mnist_data()
    train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
    test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)
    network.fit(train_images, train_labels, batch_size, epochs)
    test_loss, test_acc = network.evaluate(test_images, test_labels)
    # save model to disk
    save_model(network, out_file_name, overwrite=True, include_optimizer=True)

    print("test_loss: {}, test_acc: {}".format(test_loss, test_acc))
    session.close()

In [ ]:
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from IPython import display
from time import sleep
import numpy as np

train_images, train_labels, test_images, test_labels = get_mnist_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)
m = load_model("deep.hdf5", compile=True)
predicted_labels = m.predict(test_images, batch_size=256)

## Darstellung der Ergebnisse

In [ ]:
for i in range(len(test_labels)):
    real_value = np.argmax(test_labels[i])
    predicted_value = np.argmax(predicted_labels[i])
    if real_value != predicted_value:
        plt.title(
            "{} classified as {}".format(
                real_value,
                predicted_value,
            )
        )
        plt.imshow(test_images[i].reshape(28, 28), cmap=plt.cm.gray)
        display.clear_output(wait=True)
        display.display(plt.gcf())
        sleep(2.0)

## Trend

Es werde immer mehr vortrainierte Netze verwendet, die angepasst und dann z.B. in [OpenCV](https://docs.opencv.org/4.4.0/d6/d0f/group__dnn.html#gad820b280978d06773234ba6841e77e8d) geladen und dann direkt eingesetzt werden können.

# Quellen

1. C. Rossant: *IPython Interactive Computing and Visualization Cookbook - Second Edition*, Packt Publishing [Online (O'Reilly)](https://learning.oreilly.com/library/view/ipython-interactive-computing/9781785888632).
2. [Khronos (WebGL, Vulkan)](https://www.khronos.org).